In [3]:
import netifaces as ni
ip = ni.ifaddresses('enp7s0')[ni.AF_INET][0]['addr']
print(ip)  # should print "192.168.100.37"

192.168.1.13


In [4]:
import socket
def get_ip_address():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.connect(("8.8.8.8", 80))
    return s.getsockname()[0]

get_ip_address()

'172.16.0.2'

In [8]:
from socket import getaddrinfo, gethostname
import ipaddress

def get_ip(ip_addr_proto="ipv4", ignore_local_ips=True):
    # By default, this method only returns non-local IPv4 addresses
    # To return IPv6 only, call get_ip('ipv6')
    # To return both IPv4 and IPv6, call get_ip('both')
    # To return local IPs, call get_ip(None, False)
    # Can combine options like so get_ip('both', False)

    af_inet = 2
    if ip_addr_proto == "ipv6":
        af_inet = 30
    elif ip_addr_proto == "both":
        af_inet = 0

    system_ip_list = getaddrinfo(gethostname(), None, af_inet, 1, 0)
    ip_list = []

    for ip in system_ip_list:
        ip = ip[4][0]

        try:
            ipaddress.ip_address(str(ip))
            ip_address_valid = True
        except ValueError:
            ip_address_valid = False
        else:
            if ipaddress.ip_address(ip).is_loopback and ignore_local_ips or ipaddress.ip_address(ip).is_link_local and ignore_local_ips:
                pass
            elif ip_address_valid:
                ip_list.append(ip)

    return ip_list

print(f"Your IP address is: {get_ip()}")

Your IP address is: []


In [9]:
import socket

def get_my_ip_address(remote_server="google.com"):
    """
    Return the/a network-facing IP number for this system.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect((remote_server, 80))
        return s.getsockname()[0]
    

get_my_ip_address(remote_server="google.com")

'172.16.0.2'

In [10]:
import netifaces as ni

def test_network():
    interfaces = ni.interfaces()

    for i in interfaces: #Will cycle through all available interfaces and check each one.
        if i != "lo": #This will remove lo from the interfaces it checks.
            try:
                ni.ifaddresses(i)
                gws = ni.gateways()
                gateway = gws['default'][ni.AF_INET][0]
                ip = ni.ifaddresses(i)[ni.AF_INET][0]['addr']
                sm = ni.ifaddresses(i)[ni.AF_INET][0]['netmask']
                print ("Network information for " + i + ":")
                print ("IP address: " + ip)
                print ("Subnet Mask: " + sm)
                print ("Gateway: " + gateway)
                print ()
            except: #Error case for a disconnected Wi-Fi or trying to test a network with no DHCP
                print (i + " is not connected or DHCP is not available. Try setting a static IP address.")
test_network()

Network information for enp7s0:
IP address: 192.168.1.13
Subnet Mask: 255.255.255.0
Gateway: 192.168.1.1

wlp5s0 is not connected or DHCP is not available. Try setting a static IP address.
Network information for virbr0:
IP address: 192.168.122.1
Subnet Mask: 255.255.255.0
Gateway: 192.168.1.1

virbr0-nic is not connected or DHCP is not available. Try setting a static IP address.
Network information for CloudflareWARP:
IP address: 172.16.0.2
Subnet Mask: 255.255.255.255
Gateway: 192.168.1.1

Network information for docker0:
IP address: 172.17.0.1
Subnet Mask: 255.255.0.0
Gateway: 192.168.1.1

Network information for br-4cd9a4f278bf:
IP address: 172.22.0.1
Subnet Mask: 255.255.0.0
Gateway: 192.168.1.1

Network information for br-8fcdc0016433:
IP address: 172.18.0.1
Subnet Mask: 255.255.0.0
Gateway: 192.168.1.1

vethc02b657 is not connected or DHCP is not available. Try setting a static IP address.
vethf5d5d08 is not connected or DHCP is not available. Try setting a static IP address.
ve

In [ ]:
# ref: https://manpages.ubuntu.com/manpages/focal/man7/systemd.net-naming-scheme.7.html
nic_naming_prefix = {
    'en':'ethernet',
    'ib':'infiniBand',
    'sl':'slip', # serial line IP
    'wl':'wireless local area network',
    'ww':'wireless wide area network',
    'br': 'bridge',
    'docker': 'docker',
    'lo': 'loopback',
    'veth': 'virtual ethernet devices',
    'virbr': 'virtual bridges',
}

In [16]:
import netifaces as ni

def get_ip(nic_prefix='en'):
    interfaces = ni.interfaces()

    for i in interfaces: #Will cycle through all available interfaces and check each one.
        if i.startswith(nic_prefix): # this is the ethernet
            try:
                ni.ifaddresses(i)
                ip = ni.ifaddresses(i)[ni.AF_INET][0]['addr']
                return ip
            except: #Error case for a disconnected Wi-Fi or trying to test a network with no DHCP
                return None
get_ip()

'192.168.1.13'

In [17]:
origins_allowed = [
    "http://localhost:5173",
    #"http://192.168.1.13:5173",
]
origins_allowed.append(f'http://{get_ip()}:5173')
origins_allowed

['http://localhost:5173', 'http://192.168.1.13:5173']